In [15]:
!pip install spacy-transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [17]:
!git clone https://github.com/tulee3474/AI_Resume_Builder

Cloning into 'AI_Resume_Builder'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 273 (delta 5), reused 2 (delta 0), pack-reused 231
Receiving objects: 100% (273/273), 1.44 MiB | 6.69 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [18]:
%cd /content/AI_Resume_Builder/
!git pull

/content/AI_Resume_Builder
Already up to date.


In [20]:
import json
cv_data=json.load(open('/content/AI_Resume_Builder/Job_description_annotated/mergerd jd.json','r'))

In [21]:
!python -m spacy init fill-config /content/AI_Resume_Builder/base_config.cfg /content/AI_Resume_Builder/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/AI_Resume_Builder/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:
def get_spacy_doc(file, data):
  nlp=spacy.blank('en')
  db=DocBin()

  for item in data:
    text = item['text']
    doc=nlp.make_doc(text)
    annot=item['label']

    
    ents=[]
    entity_indices=[]

    for start, end, label in annot:
      skip_entity=False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
         continue
        
      entity_indices = entity_indices + list(range(start, end))

      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
        
      if span is None:
        err_data=str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)
        
      else:
        ents.append(span)

    try:
        doc.ents=ents
        db.add(doc)
    except:
      pass


  return db

In [23]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(cv_data,test_size=0.3)

In [24]:
len(train), len(test)

(111, 48)

In [25]:
cv_data[0]

{'id': 19,
 'label': [[0, 5, 'company'],
  [337, 357, 'project'],
  [362, 418, 'project'],
  [421, 459, 'job'],
  [475, 481, 'required skills'],
  [483, 488, 'required skills'],
  [489, 500, 'required skills'],
  [505, 513, 'required skills'],
  [642, 690, 'project'],
  [695, 772, 'project'],
  [809, 847, 'benefit'],
  [887, 927, 'benefit'],
  [989, 1057, 'benefit'],
  [1160, 1218, 'benefit'],
  [1464, 1532, 'benefit']],
 'text': "Yapz is a cutting edge, growth-stage startup that allows you to find, connect, and co-host events with clubs in your local area. In addition to their schooling, social, family and work lives, college students partake in extracurricular clubs on campus. To allocate enough time to make those clubs successful is extremely difficult. Yapz makes running a club and finding partner clubs for events easy and time efficient.\n\nWe are looking for a Backend Developer who works with Python, flask, SQLAlchemy and Postgres. The developer should also have a practical under

In [26]:
file=open('error.txt','w')
db=get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [27]:
db.tokens

[array([[ 7112935854966758681,  1049729868293614729,                    0,
         ...,                    0,                    0,
                            0],
        [17185081534114707056, 15255859468896132977,                    0,
         ...,                    0,                    0,
                            0],
        [18250316222013540736, 18250316222013540736,                    0,
         ...,                    0,                    0,
                            0],
        ...,
        [  333741129952275274,   333741129952275274,                    0,
         ...,                    0,                    0,
                            0],
        [16764210730586636600, 16764210730586636600,                    0,
         ...,                    0,                    0,
                            0],
        [ 8692143769139790023, 14840232511926861974,                    0,
         ...,                    0,                    0,
                            0

In [28]:
!python -m spacy train /content/AI_Resume_Builder/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-07 22:03:10,815] [INFO] Set up nlp object from config
[2022-08-07 22:03:10,825] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-07 22:03:10,830] [INFO] Created vocabulary
[2022-08-07 22:03:10,831] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 479kB/s]
Downloading: 100% 878k/878k [00:01<00:00, 813kB/s] 
Downloading: 100% 446k/446k [00:00<00:00, 496kB/s]
Downloading: 100% 1.29M/1.29M [00:01<00:00, 1.22MB/s]
Downloading: 100% 478M/478M [00:06<00:00, 73.2MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with a

In [29]:
nlp=spacy.load('/content/AI_Resume_Builder/output/model-best')

In [30]:
doc=nlp('Seasoned and independent Front End Developer with 5 years of experience in blending the art of design with skill of programming to deliver an immersive and engaging user experience through efficient website development, proactive feature optimization, and relentless debugging.')
for ent in doc.ents:
  print(ent.text, "    ->>>> ", ent.label_)

In [31]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 3.8 MB/s 


In [32]:
import sys, fitz

In [44]:
for x in range(1,11):
  fname='/content/AI_Resume_Builder/jd test/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('num '+str(x)+' resume test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

num 1 resume test
Django Developer  ->>>>>>  job
Exposure to Scrum or Agile software development processes  ->>>>>>  required skills
Self managing, comfortable with remote working Desire to learn  ->>>>>>  required skills
1+ years  ->>>>>>  required year
Python software development  ->>>>>>  required skills
2+ years  ->>>>>>  required year
any modern language  ->>>>>>  required skills
Web Frameworks and template languages  ->>>>>>  required skills
Flask  ->>>>>>  required skills
Django  ->>>>>>  required skills
Frontend skills like HTML, CSS, Javascript, JQuery, Bootstrap, UI Design Exposure / deployment to Amazon Web Services  ->>>>>>  required skills
Computers  ->>>>>>  required skills


num 2 resume test
DevOps Engineer  ->>>>>>  job
Kubernetes  ->>>>>>  preferred skills
Docker  ->>>>>>  preferred skills
AWS  ->>>>>>  preferred skills
Jenkins  ->>>>>>  required skills
Linux  ->>>>>>  required skills
Docker  ->>>>>>  required skills
AWS  ->>>>>>  required skills
Kubernets  ->>>>>>  r